-----------------------
#### basic chain
--------------------------

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain_openai  import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
# Define the prompt template
prompt = PromptTemplate.from_template("What is the capital of {country}?")

In [3]:
# Initialize the LLM and chain
llm   = ChatOpenAI()

chain = LLMChain(llm   = llm, 
                 prompt= prompt)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_14080\2838644879.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm   = llm,


In [4]:
# Run the chain
result = chain.invoke({"country": "France"})
print(result)  # Output should be "Paris"

{'country': 'France', 'text': 'The capital of France is Paris.'}


example

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

In [15]:
product = "Nexon"

In [16]:
# Run the chain
result = chain.invoke({"product": product})
print(result)  

{'product': 'Nexon', 'text': 'Nexon Innovations'}


**Simple sequential chain**

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
llm_model = 'gpt-3.5-turbo'

In [17]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [18]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [19]:
overall_simple_chain = SimpleSequentialChain(chains =[chain_one, chain_two],
                                             verbose=True
                                            )

In [20]:
product = "fitbit"

In [21]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
ActiveLife Technologies
ActiveLife Technologies is a leading provider of innovative health and wellness products, promoting an active lifestyle for improved well-being.

> Finished chain.


{'input': 'fitbit',
 'output': 'ActiveLife Technologies is a leading provider of innovative health and wellness products, promoting an active lifestyle for improved well-being.'}

**Exercise on Simple Sequential chain**

- Text Processing Pipeline

    - Remove punctuation from a text.
    - Convert the text to lowercase.
    - Count the number of words in the text.

In [39]:
# Remove punctuation
prompt_remove_punctuation = PromptTemplate(
    input_variables = ["text"],
    template        = "Remove all punctuation from the following text:\n{text}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt_remove_punctuation)

In [40]:
# Convert to lowercase
prompt_to_lowercase = PromptTemplate(
    input_variables = ["text"],
    template        = "Convert the following text to lowercase:\n{text}"
)

# Chain 2
chain_two = LLMChain(llm=llm, prompt=prompt_to_lowercase)

In [41]:
# Count the words
prompt_count_words = PromptTemplate(
    input_variables =["text"],
    template        = "Count the number of words in the following text and return only the number:\n{text}"
)

# Chain 3
chain_three = LLMChain(llm=llm, prompt=prompt_count_words)

In [42]:
overall_simple_chain = SimpleSequentialChain(chains =[chain_one, chain_two, chain_three],
                                             verbose=True
                                            )

In [43]:
text = 'Modern humans arrived on the Indian subcontinent'

overall_simple_chain.invoke(text)



> Entering new SimpleSequentialChain chain...
Modern humans arrived on the Indian subcontinent
modern humans arrived on the indian subcontinent
7

> Finished chain.


{'input': 'Modern humans arrived on the Indian subcontinent', 'output': '7'}

#### Sequential chains

In [44]:
from langchain.chains.sequential import SequentialChain

In [45]:
# Define the prompt templates
prompt1 = PromptTemplate.from_template("What is the capital of {country}?")
prompt2 = PromptTemplate.from_template("Describe the main attractions in {city}.")

In [46]:
# Initialize the language model
llm = ChatOpenAI()

In [47]:
# Create individual chains with explicit output
chain1 = LLMChain(llm       = llm, 
                  prompt    = prompt1, 
                  output_key="city")               # Sets "city" as output for chain1

chain2 = LLMChain(llm       = llm, 
                  prompt    = prompt2, 
                  output_key= "city_description")  # Sets "city_description" as output for chain2

In [49]:
# Set up the SequentialChain
sequential_chain = SequentialChain(
    chains           = [chain1, chain2],
    input_variables  = ["country"],          # Input to the first chain
    output_variables = ["city_description"]  # Output from the last chain
)

In [50]:
# Run the SequentialChain
result = sequential_chain.invoke({"country": "Japan"})
print(result["city_description"])  # Should describe attractions in the capital city

Tokyo, the bustling capital of Japan, is a vibrant city with a plethora of attractions to offer visitors. Some of the main attractions in Tokyo include:

1. Tokyo Tower: A symbol of the city, this iconic red and white tower offers panoramic views of Tokyo from its observation decks.

2. Meiji Shrine: A peaceful oasis in the heart of the city, this Shinto shrine is dedicated to Emperor Meiji and Empress Shoken.

3. Tsukiji Fish Market: One of the largest fish markets in the world, Tsukiji is a must-visit for seafood lovers looking to sample fresh sushi and seafood.

4. Shibuya Crossing: Known as the busiest pedestrian crossing in the world, Shibuya Crossing is a must-see for visitors looking to experience the chaos and energy of Tokyo.

5. Harajuku: A vibrant district known for its quirky fashion, Harajuku is a popular shopping destination with trendy boutiques, cafes, and street food stalls.

6. Shinjuku Gyoen National Garden: A tranquil escape from the hustle and bustle of the city, t

**Exercise on Sequential chain with multiple input variables**

Design a Sequential Chain to analyze medical data from a patient's record. The chain should:

- Take multiple inputs: age, blood_pressure, cholesterol_level, and smoking_status.

    - Step 1: Assess the patient's `risk level` for heart disease based on the inputs (e.g., Low, Moderate, High).
    - Step 2: Suggest `lifestyle changes` based on the risk level and smoking status.
    - Step 3: Provide a `follow-up recommendation` based on the risk level and age.

In [51]:
# Step 1: Assess heart disease risk level
prompt_risk_assessment = PromptTemplate(
    input_variables = ["age", "blood_pressure", "cholesterol_level", "smoking_status"],
    template        = (
        "Based on the following information:\n"
        "- Age: {age}\n"
        "- Blood Pressure: {blood_pressure}\n"
        "- Cholesterol Level: {cholesterol_level}\n"
        "- Smoking Status: {smoking_status}\n"
        "Assess the risk level for heart disease as Low, Moderate, or High."
    )
)

In [52]:
# Step 2: Suggest lifestyle changes
prompt_lifestyle_changes = PromptTemplate(
    input_variables = ["risk_level", "smoking_status"],
    template        = (
        "Given that the heart disease risk level is {risk_level} and the patient is a "
        "{smoking_status} smoker, suggest lifestyle changes to improve health."
    )
)

In [53]:
# Step 3: Provide follow-up recommendation
prompt_follow_up = PromptTemplate(
    input_variables = ["risk_level", "age"],
    template = (
        "For a patient with a heart disease risk level of {risk_level} and age {age}, "
        "provide a follow-up recommendation (e.g., frequency of doctor visits or specific tests)."
    )
)

In [56]:
# Create individual chains with explicit output
chain1 = LLMChain(llm       = llm, 
                  prompt    = prompt_risk_assessment, 
                  output_key="risk_level")              

chain2 = LLMChain(llm       = llm, 
                  prompt    = prompt_lifestyle_changes, 
                  output_key= "lifestyle_changes")  

chain3 = LLMChain(llm       = llm, 
                  prompt    = prompt_lifestyle_changes, 
                  output_key= "recommendation")  

In [57]:
# Set up the SequentialChain
sequential_chain = SequentialChain(
    chains           = [chain1, chain2, chain3],
    input_variables  = ["age", "blood_pressure", "cholesterol_level", "smoking_status"],         
    output_variables = ["recommendation"]         # Output from the last chain
)

In [59]:
# input data
patient_data = {
    "age": 43,
    "blood_pressure": "130/100",
    "cholesterol_level": "200 mg/dL",
    "smoking_status": 'current'
}


In [61]:
# Run the SequentialChain
result = sequential_chain.invoke(patient_data)
print(result["recommendation"])

1. Quit smoking: Quitting smoking is one of the most important steps you can take to improve your heart health. Smoking damages your blood vessels, increases your blood pressure, and raises your risk of heart disease. Talk to your healthcare provider about resources and support to help you quit smoking.

2. Eat a heart-healthy diet: Focus on eating a diet rich in fruits, vegetables, whole grains, lean proteins, and healthy fats. Limit your intake of saturated and trans fats, cholesterol, and sodium. Eating a balanced diet can help lower your cholesterol levels and blood pressure.

3. Exercise regularly: Aim for at least 150 minutes of moderate-intensity exercise per week, such as brisk walking, cycling, or swimming. Regular physical activity can help lower your blood pressure, improve your cholesterol levels, and maintain a healthy weight.

4. Manage stress: Chronic stress can contribute to high blood pressure and heart disease. Practice stress-reducing activities such as meditation, d